In [47]:
import sys
import requests
import pandas as pd 

sell_SL_diff_7d = []
sell_SL_diff_1d = []
sell_sold_5pct_1d = []
profit_est = []

url_items_tp_data = (   'https://api.datawars2.ie/gw2/v1/items/json?fields='
                        'id,name,buy_price,sell_price,roi,profit,'
                        '7d_buy_listed,7d_buy_sold,7d_sell_listed,7d_sell_sold,'
                        '1d_buy_listed,1d_buy_sold,1d_sell_listed,1d_sell_sold')


result = requests.get(url_items_tp_data)

if(result.status_code == 200):
    data = result.json()
    df = pd.json_normalize(data)

else:
	sys.exit("Status code not 200")



for i, row in df.iterrows():

    sell_SL_diff_7d.append(row['7d_sell_sold'] - row['7d_sell_listed'])
    sell_SL_diff_1d.append(row['1d_sell_sold'] - row['1d_sell_listed'])
    sell_sold_5pct_1d.append(row['1d_sell_sold'] * 0.05)
    profit_est.append(row['profit'] * row['1d_sell_sold'] * 0.05)

df['sell_SL_diff_7d'] = sell_SL_diff_7d
df['sell_SL_diff_1d'] = sell_SL_diff_1d
df['ss_speed'] = sell_sold_5pct_1d
df['profit_est'] = profit_est

df['roi'] = df['roi'] * 100

# df.round()
# df.round({"roi":2})

df = df.round(decimals=0).astype(object)
df = df.convert_dtypes()

# df.dtypes

In [51]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
	display(df[[	
		'name',
		'sell_price',
		'buy_price',
		'profit',
		'profit_est',
		'roi',


		'7d_buy_listed',
		'1d_buy_listed',
		'7d_buy_sold',
		'1d_buy_sold',
		# '7d_sell_listed',
		# '1d_sell_listed',
		'7d_sell_sold',
		'1d_sell_sold',

		'sell_SL_diff_7d',
		'sell_SL_diff_1d',
		'ss_speed'
		]]

	.loc	[
		(df['profit'] > 10) &
		(df['roi'] > 1) &
		(df['7d_sell_sold'] > 500) &
		(df['7d_buy_sold'] > 500) &
		(df['sell_SL_diff_7d'] > 0) #&
		]

	.sort_values(by='profit_est', ascending=False)

	# .head(30)

	.style
		.hide_index()
		# .set_properties(**{'background-color': 'red'}, subset=['roi'])
		.bar(subset=[	'7d_buy_listed',
				'1d_buy_listed',
				'7d_buy_sold',
				'1d_buy_sold',
				# '7d_sell_listed',
				# '1d_sell_listed',
				'7d_sell_sold',
				'1d_sell_sold',], color=['#d65f5f', '#5fba7d'])
	# .shape[0]
	)

name,sell_price,buy_price,profit,profit_est,roi,7d_buy_listed,1d_buy_listed,7d_buy_sold,1d_buy_sold,7d_sell_sold,1d_sell_sold,sell_SL_diff_7d,sell_SL_diff_1d,ss_speed
Superior Sharpening Stone,1050,816,77,16721,9,10580,4104,8579,700,33194,4343,1292,-1651,217
Master Maintenance Oil,1182,805,200,15790,25,6133,582,3412,429,10110,1579,2268,-15,79
Sheet of Coarse Paper,394,183,152,13315,83,4128,1250,3711,2709,11009,1752,28,680,88
Thin Leather Strap,225,132,59,11552,45,4821,2249,3837,516,4611,3916,3762,3579,196
Rugged Leather Strap,602,403,109,10595,27,2096,350,1905,68,2398,1944,1645,1782,97
Furious Sharpening Stone,3100,2243,392,9702,17,1333,729,1226,109,4914,495,217,-64,25
Mithril Axe Blade,887,405,349,7975,86,3455,560,1640,0,4179,457,137,-912,23
Peach Tart,258,85,134,6298,158,2239,251,2254,202,3994,940,824,734,47
Corrupted Shard,493,271,148,6186,55,3735,751,2620,362,4888,836,695,-24,42
Master Tuning Crystal,1620,1339,38,5293,3,3319,751,1930,236,18198,2786,3411,112,139
